# Portfolio Optimization

In [8]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize
import yfinance as yf

In [5]:
# Function to read data and calculate daily returns
raw_data_path = '../data/raw/'
processed_data_path = '../data/processed/'
suffix = 'one_year_data.csv'
stocks = ['AAPL', 'AMZN', 'EEM', 'GLD', 'JNJ', 'JPM', 'SPY', 'TSLA']
file_paths = [f"{raw_data_path}{stock}_{suffix}" for stock in stocks]

In [14]:
def load_data(files):
    data = {}
    for file in files:
        df = pd.read_csv(file, index_col='Date', parse_dates=True)
        data[file.split('/')[-1].split('_')[0]] = df['Close']
    data = pd.DataFrame(data)
    data.to_csv(processed_data_path + 'one_year_price.csv')
    return data

def get_current_tbill_rate():
    # Fetching the 3-month T-Bill yield
    tbill = yf.Ticker("^IRX")  # 13-week Treasury bill
    hist = tbill.history(period="1d")
    current_yield = hist['Close'].iloc[-1]  # Get the last closing yield
    return current_yield / 100  # Convert to a decimal format

# Calculate portfolio performance metrics
def portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate=0.01):
    returns = np.sum(mean_returns * weights) * 252
    std_dev = np.sqrt(np.dot(weights.T, np.dot(cov_matrix, weights))) * np.sqrt(252)
    sharpe_ratio = (returns - risk_free_rate) / std_dev
    return std_dev, returns, sharpe_ratio

def check_sum(weights):
    return np.sum(weights) - 1  # Weights must sum to 1

constraints = ({'type': 'eq', 'fun': check_sum})

def negative_sharpe_ratio(weights, mean_returns, cov_matrix, risk_free_rate):
    return -portfolio_performance(weights, mean_returns, cov_matrix, risk_free_rate)[2]

def maximize_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate=0.01):
    num_assets = len(mean_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    bounds = tuple((-1, 1) for asset in range(num_assets))
    initial_guess = num_assets * [1./num_assets,]
    result = minimize(negative_sharpe_ratio, initial_guess, args=args, method='SLSQP', bounds=bounds, constraints=constraints)
    return result

risk_free_rate = get_current_tbill_rate()
print(f"Current 3-month T-Bill rate: {risk_free_rate:.4%}")
# Current 3-month T-Bill rate: 5.2450% (last run)

Current 3-month T-Bill rate: 5.2450%


In [12]:
# There are 249 pieces of data from 2023-04-01 to 2024-04-01
price_data = load_data(file_paths)
returns = price_data.pct_change().dropna()
mean_returns = returns.mean()
cov_matrix = returns.cov()
risk_free_rate = 0.05245

In [24]:
optimal_portfolio = maximize_sharpe_ratio(mean_returns, cov_matrix, risk_free_rate)
portfolio_volatility = None
mu = None
if optimal_portfolio.success:  # Check if the optimization was successful
    print("Optimal portfolio list:", stocks)
    print("Optimal portfolio weights:", optimal_portfolio.x)
    vol, ret, sharpe = portfolio_performance(optimal_portfolio.x, mean_returns, cov_matrix, risk_free_rate)
    portfolio_volatility = vol
    mu = ret
    print("Expected annual return:", ret)
    print("Volatility:", vol)
    print("Sharpe Ratio:", sharpe)
else:
    print("Optimization did not converge:", optimal_portfolio.message)

Optimal portfolio list: ['AAPL', 'AMZN', 'EEM', 'GLD', 'JNJ', 'JPM', 'SPY', 'TSLA']
Optimal portfolio weights: [-0.31147067  0.27887588 -0.80288713  0.40674235 -0.13353665  0.65459742
  0.99902492 -0.09134612]
Expected annual return: 0.7156253001996112
Volatility: 0.16620457101306954
Sharpe Ratio: 3.9901146960144165


In [25]:
weights = np.array([-0.31147067, 0.27887588, -0.80288713, 0.40674235, -0.13353665, 0.65459742, 0.99902492, -0.09134612])  # Our portfolio weights
current_prices = price_data.iloc[-1, :]
current_portfolio_value = np.dot(current_prices, weights)
expected_annual_return = mu
print("Current Portfolio Value:", current_portfolio_value)
print("Portfolio Volatility:", portfolio_volatility)
print("Expected annual return:", expected_annual_return)


Current Portfolio Value: 663.3186318547223
Portfolio Volatility: 0.16620457101306954
Expected annual return: 0.7156253001996112


# Simulation

### GBM

### Merton

### CEV

### Heston